# 3F7 Full Technical Report

**By Diego García Medina**

The code below will improve on the arithmetic coding by using conditional probabilities, given by a pre-trained LSTM network on English language.

In [1]:
f = open('hamlet.txt')
hamlet = open('pride_and_prejudice.txt').read() + f.read()
f.close()
print(hamlet[:294])

The Project Gutenberg EBook of Pride and Prejudice, by Jane Austen

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or onlin


In [2]:
from itertools import groupby
frequencies = dict([(key, len(list(group))) for key, group in groupby(sorted(hamlet))])
Nin = sum([frequencies[a] for a in frequencies])
p = dict([(a,frequencies[a]/Nin) for a in frequencies])

print(f'File length: {Nin}')
print("There are {} characters in the alphabet".format(len(frequencies)))
print("The characters are the following:\n")
for char in frequencies.keys():
    print(char)

File length: 911214
There are 86 characters in the alphabet
The characters are the following:



 
!
"
#
$
%
&
'
(
)
*
,
-
.
/
0
1
2
3
4
5
6
7
8
9
:
;
?
@
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z
[
]
_
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z
|


In [3]:
method = 'conditional-arithmetic'
filename = 'hamlet.txt'

In [4]:
from camzip import camzip
from camunzip import camunzip
#camzip(method, filename)
camunzip(filename + '.cz' + method[0])

Using TensorFlow backend.
C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression ratio (rateless): {Nout/Nin}')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
with open(filename + '.czp', 'r') as fp:
    freq = load(fp)
pf = dict([(a, freq[a]/Nin) for a in freq])
print(f'Entropy: {H(pf)} bits per symbol')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

NameError: name 'stat' is not defined